In [1]:
import os
from util import Stats
import pandas as pd
from util import count_lines_from_file, collect_all_seeds


LANGS = open("./meta/low-langs.txt").read().split()
DATA_DIR = "./data"

records = []

for lang in LANGS:
    stats = Stats(os.path.join(DATA_DIR, f"{lang}_en"))
    record_1, record_2 = stats.analyze()
    records += [record_1, record_2]
    
df = pd.DataFrame(records)

In [2]:
def get_lang(row):
    langs = row.data_dir.split('/')[-1].split('_')
    if row.lang == 1:
        return langs[-2]
    else:
        return langs[-1]

df['language'] = df.apply(get_lang, axis=1)

In [3]:
ILL_stats = {}
ILL_stats['num_of_ILLs'] = count_lines_from_file("data/zh_yue_en/ref_ent_ids")
ILL_stats['train'] = count_lines_from_file("data/zh_yue_en/train")
ILL_stats['dev'] = count_lines_from_file("data/zh_yue_en/dev")
ILL_stats['test'] = count_lines_from_file("data/zh_yue_en/test")

print(ILL_stats)

{'num_of_ILLs': 15000, 'train': 9450, 'dev': 1050, 'test': 4500}


In [4]:
table = df[(df.lang == 1) | (df.index == 1)] \
    .drop(columns=["lang", "data_dir"]) \
    .sort_values(by="num_entities", ascending=False) \
    .transpose() \
    .sort_index()

LANGS = open("meta/low-langs.txt").read().strip().split()
SEED_ID = "./meta/seeds.id.{lang}.txt"

seed_path_map = {lang: SEED_ID.format(lang=lang) for lang in LANGS}
seeds = collect_all_seeds(seed_path_map)
seeds["yue"] = seeds.pop("zh_yue")

count_seeds = lambda ents: len(ents)
num_of_seeds = {lang: count_seeds(ents) for lang, ents in seeds.items()}
num_of_seeds = pd.DataFrame([num_of_seeds]).rename(index={0: "num_of_seeds"})

miss = lambda ents: round((len(ents) - sum(ents.values()))/len(ents), 2)
missing_rate = {lang: miss(ents) for lang, ents in seeds.items()}
missing_rate = pd.DataFrame([missing_rate]).rename(index={0: "ILL_missing_rate"})

table = table.rename(columns = dict(table.iloc[0]))
table = table.drop(index="language")
table = pd.concat([table, num_of_seeds, missing_rate])

In [5]:
table

,en,fa,ko,he,hy,el,yue,ka,mk,bn,ur,hi,ta,tl,ml,kn
num_attribute_triples,1473285,80602,71547,121470,69381,94136,41823,67154,108720,71699,34436,31804,32513,22315,52570,35089
num_attributes,4634,1035,1056,984,175,627,80,224,1663,685,231,77,133,28,130,56
num_descriptions,411872,34608,34484,33896,33408,32899,29806,29140,28024,25161,24311,22606,22522,21258,21231,17071
num_entities,411882,34635,34489,33899,33438,32912,29893,29157,28043,25199,24331,22631,22541,21271,21243,17081
num_relation_triples,1083093,58882,57774,70089,65121,67831,49155,53279,52703,49681,36980,36101,42762,33554,38270,24849
num_relations,921,401,438,468,236,389,117,255,433,278,199,128,193,62,157,92
num_of_seeds,149369,15000,15000,15000,15000,15000,15000,15000,15000,15000,15000,15000,15000,15000,15000,15000
ILL_missing_rate,0,0.14,0.31,0.27,0.38,0.26,0.27,0.25,0.29,0.16,0.13,0.39,0.37,0.16,0.26,0.38


#### Sampling methods
(1) Languages: 15 languages, each of which uses its distinct script.

    Rank - Lang (Script; Wikicode)
    
    <Wikipeida #article rank: 26-50>
    26 - Hebrew (Hebr; he), 
    28 - Korean (Hang; ko), 
    37 - Farsi (Persian; fa), 
    44 - Greek (Grek; el), 
    50 - Georgian (Geor; ka)

    <Wikipeida #article rank: 51-75>
    54 - Hindi (Deva; hi), 
    60 - Macedonian (Cyrl; mk), 
    61 - Tagalog (Latn; tl), 
    63 - Bengali (Beng; bn), 
    67 - Tamil (Taml; ta)

    <Wikipeida #article rank: 76-108>
    83 - Cantonese (Hant; yue), 
    86 - Urdu (Arab; ur), 
    90 - Malayalam (Mlym; ml), 
    99 - Kannada (Knda; kn), 
    108 - Armenian (Armn; hy) 

(2) We first sample 15K seed entities from each language, except for English.

(3) To ensure comparibility, all the sampled entities that have English counterparts, i.e., the union, are set as English seeds.
As a result, there are approximately 150K English seed entities. 

(4) Using the seeds of each language, we collect the one-hop neighbours to build the monolingual KG, which triples are inserted if:
    
    (i) the relation has a label written in this language, and 
    (ii) the object has a wiki sitelink of this language.
    
(4) Collect language-specific attributes of the seeds, which labels are written in the specified language.

(5) Collect language-specific descriptions of all the entities in the monolingual KG of the specified language. 

(6) Note that the sampled 15K entities may and may not have ILLs to English. Each language has a different missing rate, and the missed ones are annotated in the groundtruth.  

#### Issues

(1) The union English KG is quite big. We may need to filter out some triples, e.g., low contribution ones, so a GPU has enough memory to run GCNs with it.

In [6]:
# print(pd.DataFrame.to_markdown(table))